In [ ]:
import numpy as np
import cv2 as cv
import glob



################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (5,5)
frameSize = (5312,2988)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 10
objp = objp * size_of_chessboard_squares_mm


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


images = glob.glob('C:/Users/gbo10/OneDrive/research/italy/torino/computer vision/images for calibration/*.jpg')

img_counter = 0
for image in images:

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        img_counter += 1
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.namedWindow('img', cv.WINDOW_NORMAL)
        cv.imshow('img', img)
        cv.waitKey(5000)
        cv.destroyAllWindows()



cv.destroyAllWindows()


ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

print("Camera Matrix:\n")
print(cameraMatrix)
print("\n")
print("Distortion Coefficients:\n")
print(dist)
print("\n")
print("Rotation Vectors:\n")
print(rvecs)
print("\n")
print("Translation Vectors:\n")
print(tvecs)
print("\n")
print("Number of images used for calibration: ", img_counter)
#save the camera matrix and distortion coefficients to a file


In [ ]:
np.savez("src/measurementscalibration.npz", cameraMatrix=cameraMatrix, dist=dist, rvecs=rvecs, tvecs=tvecs,ret=ret,img_counter=img_counter)

In [ ]:
#save all to csv file
import csv

with open('src/measurementscalibration.csv', mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(["Camera Matrix"])
    writer.writerow(cameraMatrix)
    writer.writerow(["Distortion Coefficients"])
    writer.writerow(dist)
    writer.writerow(["Rotation Vectors"])
    writer.writerow(rvecs)
    writer.writerow(["Translation Vectors"])
    writer.writerow(tvecs)
    writer.writerow(["Number of images used for calibration: ", img_counter])
    #projection error
    writer.writerow(["Projection Error"])
    writer.writerow(ret)


In [ ]:






############## CALIBRATION #######################################################




############## UNDISTORTION #####################################################

img = cv.imread('C:\Users\gbo10\OneDrive\research\italy\torino/computer vision/test/images/GX010097_MP4-29_jpg.rf.4f866c5d8b12dbf51b8e1307c87e4447.jpg')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))



# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)



# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)




# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print("total error: {}".format(mean_error/len(objpoints)))

import matplotlib.pyplot as plt

# Plotting the reprojection error
plt.plot(range(len(objpoints)), [cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2) for i in range(len(objpoints))])
plt.xlabel('Image Index')
plt.ylabel('Reprojection Error')
plt.title('Reprojection Error for Camera Calibration')
plt.show()

